<a href="https://colab.research.google.com/github/nickprock/corso_data_science/blob/devs/machine_learning_pills/01_supervised/03_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classificazione Multilabel

### Decision tree

In questo notebook vedremo quella che è forse la tecnica, se non più comune, più conosciuta per la classificazione, i **decision tree**.

Questa tecnica è molto flessibile e potente, permette di catturare pattern ricorrenti da set di dati anche molto etogenei.

<br>

![dt](https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2Ff%2Fff%2FDecision_tree_model.png&f=1&nofb=1)

<br>

[Image Credits](https://it.wikipedia.org/wiki/Albero_di_decisione)

<br>

Ogni nodo interno rappresenta una variabile, un arco verso un nodo figlio rappresenta un possibile valore per quella proprietà e una foglia il valore predetto per la variabile obiettivo a partire dai valori delle altre proprietà, che nell'albero è rappresentato dal cammino (path) dal nodo radice (root) al nodo foglia.

### Dataset

Il dataset utilizzato è il più famoso tra tutti e ogni corso di data scince ha almeno un esempio dove viene utilizzato: **Iris dataset**.

Iris è un dataser di 150 osservazioni e 5 feturese:
* sepal length
* sepal width
* petal length
* petal width
* species: variabile target

La variabile target è categorica, può assumere tre valori:
* setosa
* virginica
* versicolor

Le tre classi sono perfettamente bilanciate, 50 osservazioni per classe. Le restanti 4 features sono numeriche continue.

<br>

![iris](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.STg6q7XX61Tox0fuGNwvRwHaFj%26pid%3DApi&f=1)

<br>

[Image Credits](https://www.panoramio.com/)

<br>

### Load Data

Il dataset Iris è tra quelli disponibili in scikit-learn.

In [0]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [0]:
iris = load_iris()

In [0]:
iris

In [0]:
iris.data

In [0]:
iris.target

Per semplicità utilizziamo solo due delle 4 fetures.

In [0]:
x = iris.data[:, :2]
y = iris.target

In [0]:
import matplotlib.pyplot as plt

colors = ("red", "green", "blue")
groups = set(y)

plt.scatter(x[:,0], x[:,1])
plt.xlabel('petal length')
plt.ylabel('petal width')
plt.title('Iris')

#### Visualizzare il decision tree

Per mettere su un grafico l'albero generato abbiamo bisogno di una libreria apposita *export_graphviz*.

Vediamo un breve esempio con un albero semplice.

In [0]:
tree_clf = DecisionTreeClassifier(max_depth=2) # solo due livelli (più nodo root)
tree_clf.fit(x, y) # in questo caso alleniamo sull'intero dataset

In [0]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file = image_path("YOURPATH/image.dot"),
    feature_names = iris.feature_names[:2],
    class_names = iris.class_names,
    rounded = True,
    filled = True
)

# successivamente bisogna convertire il .dot in un altro formato immagine

![dt_iris](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.stack.imgur.com%2FJQpzN.png&f=1&nofb=1)

Si può notare che tutti i nodi hanno solo due foglie, questo succede perchè Scikit-learn usa l'algortimo CART che produce split binari. Altri algoritmi tipo ID3 invece possono produrre anche split con più di due nodi foglia.

La scelta della variabile viene effettuata in base alla "*purezza*" del nodo.

Per calcolare la purezza viene usato l'indice di Gini (non è l'unica metrica possibile), si dice che un nodo (variabile) è puro quando ha coefficiente zero, quindi tutte le osservazioni in ingresso in quel nodo verranno catalogate in una sola classe.

Ad esempio in questo esercizio succede a Iris Setosa.

In [0]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 42)

In [0]:
tree_clf.fit(train_x, train_y)

In [0]:
yhat = tree_clf.predict(test_x)

### Valutazione del modello

Abbiamo usato un modello senza impostare altri iperparametri se non la profondità dell'albero, valutiamo insieme il risultato.

In [0]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("confusion_matrix: ", "\n",confusion_matrix(test_y, yhat))
print("\n")
print("accuracy: ", accuracy_score(test_y, yhat))
print("\n")
print(classification_report(test_y, yhat))

#### Tuning degli iperparametri

Proviamo a fare tuning degli iperparametri del modello. Proveremo in particolare:
* cambiare la metrica, da Gini a Entropy
* rendere l'albero più profondo

In [0]:
tree_clf_2 = DecisionTreeClassifier(criterion='entropy', max_depth = 5)

In [0]:
tree_clf_2.fit(train_x, train_y)
yhat_2 = tree_clf_2.predict(test_x)

### Valutazione dei risultati

In [0]:
print("confusion_matrix: ", "\n",confusion_matrix(test_y, yhat_2))
print("\n")
print("accuracy: ", accuracy_score(test_y, yhat_2))
print("\n")
print(classification_report(test_y, yhat_2))

#### Tuning del modello

Abbiamo fatto un diverso tuning del modello in maniera manuale.

Scikit-learn ci fornisce alcuni strumenti per automatizzare questi passaggi. Il più semplice è la *Grid Search*.

Essa non è altro che una serie di parametri che vengono impostati e il modello "prova" iterativamente. Il processo è dispendioso quindi vi consiglio di approfondire anche strimenti più avanzati.

#### Cross-Validation

<br>

![CV](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fscikit-learn.org%2Fstable%2F_images%2Fgrid_search_cross_validation.png&f=1&nofb=1)

<br>

[Image Credits](https://scikit-learn.org/stable/modules/cross_validation.html)

<br>

La cross-validation è una tecnica che mira a costruire stimatori più robusti diminuendo l'errore dovuto al campionamento.

Il modello non viene allenato su un unico set di dati di train ma lo stesso viene suddiviso in N parti (fold) e viene eseguito l'allenamento N volte su N-1 folds mentre l'ennesimo farà da validiation set.

In questo modo ci si aspetta di avere un miglior tuning dei parametri, soprattutto per dataset di piccole dimensioni.

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
tuned_params = [{'criterion':['gini','entropy'], 'max_depth':[2, 5, 10], 'min_samples_split':[2, 5, 10], 'splitter':['best', 'random']}]

In [0]:
grid_tree_clf = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=tuned_params, n_jobs=-1)

In [0]:
grid_tree_clf.fit(train_x, train_y)

In [0]:
yhat_grid = grid_tree_clf.predict(test_x)

### Valutazione dei risultati

In [0]:
print("confusion_matrix: ", "\n",confusion_matrix(test_y, yhat_grid))
print("\n")
print("accuracy: ", accuracy_score(test_y, yhat_grid))
print("\n")
print(classification_report(test_y, yhat_grid))

### Esercizi

1. Colorare lo scatterplot secondo l'etichetta del target
2. Commentare i risultati
3. Provare il modello con tutte e 4 le variabili indipendenti

### Link Utili

[A Simple Explanation of Gini Impurity](https://victorzhou.com/blog/gini-impurity/)

[Entropy](https://towardsdatascience.com/entropy-how-decision-trees-make-decisions-2946b9c18c8)

[CART](https://machinelearningmastery.com/classification-and-regression-trees-for-machine-learning/)

[ID3](https://medium.com/machine-learning-guy/an-introduction-to-decision-tree-learning-id3-algorithm-54c74eb2ad55)

[Cross-Validation](https://scikit-learn.org/stable/modules/cross_validation.html)

